In [2]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
import altair as alt 
alt.data_transformers.enable('json') # work-around to let Altair handle larger data sets

#import matplotlib.pyplot as plt

DataTransformerRegistry.enable('json')

## Chargement des données aggregées

In [4]:
# loading new dataframe clean and tunned 
df=pd.read_csv("../data/baby_years.csv", sep=';')
df.shape

(249113, 4)

##### Besoin de definir des dictionnaires de ranking sur All year  --> Non present dans la dataFrame aggregée

In [5]:
# faire un ranking des prenoms sur toutes les années qui sera utile pour le trie "All"
#

d_freq = {}
d_births = {}
for key, gp in df.groupby('name'):
    d_freq[key] = gp.freq.sum()
    d_births[key] = gp.births.sum()
# sort the dict by value then key => if Same value, sort by name!
d_freq=dict(sorted(d_freq.items(), key=lambda x: x[1], reverse=True)) 
d_births=dict(sorted(d_births.items(), key=lambda x: x[1], reverse=True))


## Fonction de synthèse 
Graphique interactive

In [8]:
# Widgets
selection=widgets.Dropdown(
    options=[None, 'Top 5', 'Top 10', 'Bottom 5', 'Bottom 10'],
    value= None,
    description='Display :',
)
year=widgets.Dropdown(
    options=[None,'All'] + sorted(list(df.year.unique())),
    value= None,
    description='Year:',
)
name = widgets.Dropdown(
    options=[None] + sorted(list(df.name.unique())),
    value=None,
    description='Name:',
)

# Drawing function
def make_chart(noms):  # -> penser à inserer traitement data dans la fonction 
    
    donnee=df[df.name.isin(noms)] # data
    selector = alt.selection_multi(fields=['name']) # filter
    # chart
    chart=alt.Chart(donnee,
             width=800,
             height=400).mark_line().encode(
                                x='year:Q',
                                y='freq:Q',
                                tooltip=['name','year','births','freq'],
                                #color = alt.Color('name:N')
                                color=alt.condition(selector, 'name:N', alt.value('lightgray'))
                            ).add_selection(selector)

    
    # Creation de l'histogram
    histo = alt.Chart(donnee, width=800, height=200).mark_bar().encode(
        x='year:Q',#alt.X('births:Q', bin=alt.Bin(maxbins=100)),
        y='births:Q',
        color='name:N'
    ).transform_filter(selector)

    return chart & histo



def visualisation(selection,annee,nom):
    # get value form selectors
    if not (selection==annee==nom): # All selector are not None=>
        if selection:
            select = selection.split(' ')
            select[1] = int(select[1])
        else:
            select = [None, None]

        # update dataframe according to filter
        if nom is None:
            if annee !='All' and select[0]=='Top':
                l_noms=df[df.year==annee].nlargest(select[1],'births', keep='all').name.values
            if annee !='All' and select[0]=='Bottom':
                l_noms=df[df.year==annee].nsmallest(select[1],'births', keep='first').name.values
            if annee =='All' and select[0]=='Top':
                l_noms=list(d_freq.keys())[:5]
            if annee =='All' and select[0]=='Bottom':
                l_noms=list(d_freq.keys())[-5:]
            if annee =='All' and select[0] is None:
                l_noms = list(df.name.unique())
        else:
            l_noms = [nom]

        #data=df[df.name.isin(l_noms)] # data
        #brush = alt.selection_multi(fields=['name']) # filter
        display(make_chart(l_noms))
    
    return None

#Running graph
box = widgets.HBox([selection, year, name]) 
out = widgets.interactive_output(visualisation, {'selection':selection, 'annee':year, 'nom':name});
display(box,out)

Output()

## Que du brouillon ci dessous -> A supprimer
Les fonctions detaillées

In [34]:
# Widgets
selection=widgets.Dropdown(
    options=[None, 'Top 5', 'Top 10', 'Bottom 5', 'Bottom 10'],
    value= None,
    description='Display :',
)
year=widgets.Dropdown(
    options=[None,'All'] + sorted(list(df.year.unique())),
    value= None,
    description='Year:',
)
name = widgets.Dropdown(
    options=[None] + sorted(list(df.name.unique())),
    value=None,
    description='Name:',
)

# Drawing function
def make_chart(noms):  # -> penser à inserer traitement data dans la fonction 
    # data selection
    donnee=df[df.name.isin(noms)] # data
    selector = alt.selection_multi(fields=['name:N']) # filter
    
    # chart
    chart=alt.Chart(donnee,
             width=800,
             height=400).mark_line(point=donnee.year.nunique()==1).encode(
                                x='year:O', #scale=alt.Scale(domain=[1920, 2020])),
                                y='freq:Q',
                                #strokeWidth=alt.condition('datum.births')
                                tooltip=['name:N','year:Q','births:Q','freq:Q'],
                                #color = alt.Color('name:N')
                                color=alt.condition(selector, 'name:N', alt.value('lightgray')),
                            ).add_selection(selector)
    # histo diversite prenoms/an
    diversity = alt.Chart(df, width=800, height=200
                         ).mark_bar().encode(x=alt.X('year:Q'),
                                            y=alt.Y('nbNameYear:Q', 
                                                    axis=alt.Axis(title='Name diversity per year', 
                                                                  titleColor='#57A44C')),
                                             tooltip=['year',"nbNameYear:Q"]
                                            ).transform_aggregate(nbNameYear='count(name)',
                                                                  groupby=["year"]
                                                                    )
    # nb naissance par an
    annualbirth = alt.Chart(df, width=800, height=200
                           ).mark_line(color='red').encode(x=alt.X('year:Q'),
                                                            y=alt.Y('birthsYear:Q', 
                                                                    axis=alt.Axis(title='sum births per year', 
                                                                                  titleColor='red')),
                                                           tooltip=['year',"birthsYear:Q"]
                                                            ).transform_aggregate(
                                                                                birthsYear='sum(births)',
                                                                                groupby=["year"]
                                                                                )
    # creation d'un plot communmix des
    down = alt.layer(diversity, annualbirth).resolve_scale(y = 'independent')
    # Creation de l'histogram
    histo = alt.Chart(donnee, width=800, height=200).mark_bar().encode(
        x=alt.X('year:Q'),
        y='births:Q',
        color='name:N'
    ).transform_filter(selector)

    return chart & histo & down



def visualisation(selection,annee,nom):
    # get value form selectors
    if not (selection==annee==nom): # All selector are not None=>
        if selection:
            select = selection.split(' ')
            select[1] = int(select[1])
        else:
            select = [None, None]

        # update dataframe according to filter
        if nom is None:
            if annee !='All' and select[0]=='Top':
                l_noms=df[df.year==annee].nlargest(select[1],'births', keep='all').name.values
            if annee !='All' and select[0]=='Bottom':
                l_noms=df[df.year==annee].nsmallest(select[1],'births', keep='all').name.values
            if annee =='All' and select[0]=='Top':
                l_noms=list(d_freq.keys())[:select[1]]
            if annee =='All' and select[0]=='Bottom':
                l_noms=list(d_freq.keys())[-select[1]:]
            if annee =='All' and select[0] is None:
                l_noms = list(df.name.unique())
        else:
            l_noms = [nom]

        #data=df[df.name.isin(l_noms)] # data
        #brush = alt.selection_multi(fields=['name']) # filter
        display(make_chart(l_noms))
    
    return None

#Running graph
box = widgets.HBox([selection, year, name]) 
out = widgets.interactive_output(visualisation, {'selection':selection, 'annee':year, 'nom':name});
display(box,out)

Output()